In [77]:
import torch
from torch.autograd import Variable
torch.__version__

'1.3.1+cpu'

### 2.3 梯度

#### 2.3.1 Variable

> 1. **`Variable` 是一个物理位置不变,里面内容不断变化的对象,而里面的内容就是tensor**
> 2. **`Variable` 会构建一个计算图,`computaional graph`用于将所有计算步骤(节点)连接起来,最后误差反向传递,一次性将所有varibale里面的梯度计算出来**
> 3. **通过`Variable.data` 输出tensor形式,通过`Variable.data.numpy()`输出numpy形式**

In [78]:
tensor = torch.FloatTensor([[1, 2], [3, 4]])
# requires_grad 是参不参与反向传播
varibale = Variable(tensor, requires_grad=True)
varibale

tensor([[1., 2.],
        [3., 4.]], requires_grad=True)

In [79]:
v_out = torch.mean(varibale ** 2)
v_out

tensor(7.5000, grad_fn=<MeanBackward0>)

In [89]:
# 模拟v_out误差反向传播
# v_out = 1/4 * sum(variable ** 2)
# 所以v_out的梯度就是, d(v_out)/d(varibale) = 1/4 * 2 * variable = varibale/2
v_out.backward(retain_graph=True) # 等价于 v_out.backward(torch.tensor(1.))
varibale.grad

tensor([[1.5000, 3.0000],
        [4.5000, 6.0000]])

### 2.3.2 梯度

#### 反向传播梯度

> 1. **在tensor里面,如果将属性requires_grad=True,则表示开始追踪其所有操作,完成后调用backend()完成所有梯度计算,最后将梯度`【累加】`到grad属性中**
> 2. **调用Tensor.detach()方法将其从追踪记录中分离出来,这样防止被追踪,也可用with torch.no_grad() (多用于模型评估,因为此时不需要计算参数的梯度)**
> 3. **`Function`是对变量操作的抽象, 和 `Tensor` 可构建一个记录整个计算过程的非循环图。Tensor.grad_fn属性记录对应的信息**

#### 雅克比矩阵(Jacobian matrix)
$$
J = 
\left(\begin{array}{ccc}
\frac{\partial y_1}{\partial x_1} & \cdots & \frac{\partial y_1}{\partial x_n} \\
\vdots & \ddots & \vdots \\
\frac{\partial y_n}{\partial x_1} & \cdots & \frac{\partial y_n}{\partial x_n}
\end{array}\right)
$$

如果 $v$ 是一个标量函数的 $l = g(\vec y)$的梯度:

$$ v = \left(\begin{array}{ccc}\frac{\partial l}{\partial y_1} & \cdots & \frac{\partial l}{\partial y_n} \end{array}\right)$$


那么根据链式法则, $l$关于$\vec x$的雅克比矩阵:


$$ 
v \cdot J = \left(\begin{array}{ccc}\frac{\partial l}{\partial y_1} & \cdots & \frac{\partial l}{\partial y_n} \end{array}\right) 
\left(\begin{array}{ccc}
\frac{\partial y_1}{\partial x_1} & \cdots & \frac{\partial y_1}{\partial x_n} \\
\vdots & \ddots & \vdots \\
\frac{\partial y_n}{\partial x_1} & \cdots & \frac{\partial y_n}{\partial x_n}
\end{array}\right) 
= \left(\begin{array}{ccc} \frac{\partial l}{\partial x_1} & \cdots & \frac{\partial l}{\partial x_n}  \end{array}\right)
$$


In [106]:
# x 为直接创建,所以没有grad_fn
# 默认为flase,通过.requires_grad_(True)来设置
x = torch.ones(2, 2, requires_grad=True)
print(x.grad_fn)

None


In [107]:
# 而此时y是通过计算得到,所以有对应的属性
y = x + 2
print(y.grad_fn.next_functions)

((<AccumulateGrad object at 0x0000023789A7CA20>, 0), (None, 0))


In [108]:
print(x.is_leaf)
print(y.is_leaf)

True
False


In [109]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


In [114]:
# z此时不是标量,因而求反向梯度时,需要传入一个同形的权重,如此时的v
v = torch.tensor([[2.0, 0.2], [0.01, 0.001]], dtype=torch.float)
z.backward(v, retain_graph=True)
print(x.grad)

tensor([2.0000e+01, 2.0000e+00, 1.0000e-01, 1.0000e-02])


In [117]:
# 使用with torch.no_grad() 屏蔽梯度计算
x = torch.tensor(1.0 ,requires_grad=True)
y1 = x ** 2
with torch.no_grad():
    # 此时这个计算是没有算进梯度计算里面
    y2 = x ** 3
y3 = y1 + y2
print(y1, y1.requires_grad)
print(y2, y2.requires_grad)
print(y3, y3.requires_grad)


tensor(1., grad_fn=<PowBackward0>)True
tensor(1.)False
tensor(2., grad_fn=<AddBackward0>)True


$y_3 = y_1 + y_2 = x^2 + x^3 $ 当 $x = 1$时, $\frac{d_y}{d_x} = 5 \cdot x = 5$,但是由于 $ y_2 = x ^ 3 $ 这步计算被with no_grad()包裹,所以$y_2$的梯度不会回传,只有$y_1$的梯度才会

In [120]:
# 如果想要修改tensor的数值,但是又不希望被autograd记录,即不会影响反向传播,可以对tensor.data进行操作
x = torch.ones(1, requires_grad=True)
print(x.data)
# x.data独立于计算图之外
print(x.data.requires_grad)

tensor([1.])
False


In [121]:
y = 2 * x
x.data *= 100 # 此处只修改值,不会记录在计算图内
y.backward()
print(x) # 修改data会影响到tensor的值
print(x.grad)

tensor([100.], requires_grad=True)
tensor([2.])
